In [ ]:
import json
from openai import OpenAI
import pandas as pd

In [ ]:
client = OpenAI()

In [ ]:
messages = [{"role": "user", "content": "Hola, como estás?"}]
response = client.chat.completions.create(model="gpt-4.1-nano", messages=messages)

In [ ]:
response.choices[0].message.content

In [ ]:
def get_store_locations(postal_codes: list[int]) -> list:
    """
    Retrieves the locations of stores near the specified postal codes.

    Args:
        postal_codes (list): A list of postal codes to search for nearby stores.

    Returns:
        list: A list of store locations found near the provided postal codes.
    """
    print(f"Function called for postal codes: {postal_codes}")
    df = pd.read_csv("../data/stores.csv")
    filtered_df = df[df["postal_code"].isin(postal_codes)]
    return list(filtered_df["store_name"].values)

In [ ]:
get_store_locations([27003, 15001, 15003, 27004])

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_store_locations",
            "description": "Retrieve the names of stores near the specified postal codes.",
            "parameters": {
                "type": "object",
                "properties": {
                    "postal_codes": {
                        "type": "array",
                        "items": {"type": "integer"},
                        "description": "List of postal codes to search for nearby stores. Example: [15003, 15704]",
                    }
                },
                "required": ["postal_codes"],
                "additionalProperties": False,
            },
        },
    }
]

In [ ]:
globals().get("get_store_locations")([27003, 15001, 15003, 27004])

In [ ]:
system_prompt = """
Eres un asistente de atención al cliente encargado de ayudar a los usuarios a localizar tiendas cercanas.

Dispones de una herramienta llamada `get_store_locations`, que permite recuperar las tiendas disponibles cerca de una ubicación. 
Esta herramienta recibe un único array con varios códigos postales, por ejemplo: [28001, 28002, 28003], y devuelve las tiendas que hay en esas zonas.

INSTRUCCIONES:

1. Si el usuario menciona una ciudad:
   - Identifica el nombre exacto de la ciudad.
   - Obtén todos los códigos postales que pertenecen a esa ciudad.
     No uses solo un código o unos pocos: asegúrate de generar la lista completa de códigos postales asociados a esa ciudad.
   - Llama a la herramienta `get_store_locations` pasando un único array con todos esos códigos postales.

2. Si el usuario menciona una ciudad y un distrito o barrio específico, asegúrate igualmente de incluir todos los códigos postales que correspondan a ese ámbito, aunque sean solo un subconjunto de la ciudad.

3. No agrupes llamadas por código individual ni repitas llamadas con arrays parciales.

4. Si el usuario pregunta algo que no está relacionado con localizar tiendas por ciudad o código postal, responde:
   "Lo siento, no puedo ayudarte con esa consulta."
"""

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "Qué tiendas tienes en Coruña?"},
]
response = client.chat.completions.create(
    model="gpt-4.1-nano", messages=messages, tools=tools, parallel_tool_calls=False
)

In [ ]:
response.choices[0].finish_reason, response.choices[0].message.tool_calls

In [ ]:
def handle_tool_calls(tool_calls: list) -> list:
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        args = json.loads(tool_call.function.arguments)

        tool = globals().get(tool_name)
        result = tool(**args) if tool else {}

        results.append(
            {
                "role": "tool",
                "content": json.dumps(result),
                "tool_call_id": tool_call.id,
            }
        )
    return results

In [ ]:
system_message = [{"role": "system", "content": system_prompt}]

In [ ]:
def chat(message, history):
    messages = system_message + history + [{"role": "user", "content": message}]
    done = False
    while not done:
        response = client.chat.completions.create(
            model="gpt-4.1-nano",
            messages=messages,
            tools=tools,
            parallel_tool_calls=False,
        )
        if response.choices[0].finish_reason == "tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

In [ ]:
import gradio as gr

gr.ChatInterface(chat, type="messages").launch(share=False)